In [1]:
import numpy as np
import pandas as pd
import pickle
import sys
import torch
import math
import os
import datetime
import pickle
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
def MAE(pred, true):
    return np.mean(np.abs(pred - true))

def MSE(pred, true):
    return np.mean((pred - true) ** 2)

def RMSE(pred, true):
    return np.sqrt(MSE(pred, true))

def MAPE(pred, true):
    return np.mean(np.abs((pred - true) / true))

def MSPE(pred, true):
    return np.mean(np.square((pred - true) / true))

def DA(pred, true):
    da = (pred[1:] - true[:-1]) * (true[1:] - pred[:-1]) > 0
    return da.mean()

def metric(pred, true):
    mae = MAE(pred, true)
    mse = MSE(pred, true)
    rmse = RMSE(pred, true)
    mape = MAPE(pred, true)
    mspe = MSPE(pred, true)
    da = DA(pred, true)
    
    return mae, mse, rmse, mape, mspe, da

In [ ]:
data_path=r'datasets/gefcom2017.csv'

df = pd.read_csv(data_path, encoding='gb18030')
# df['trend'] = range(1,len(df)+1)
# df = df[['trend','ts', 'zone', 'demand', 'drybulb', 'dewpnt', ]]
df = df[['ts', 'trend', 'zone', 'demand', 'drybulb', 'dewpnt' ]]
df['ts'] = pd.to_datetime(df['ts'])


df['drybulb^2'] = df['drybulb'].apply(lambda x: x**2)
df['drybulb^3'] = df['drybulb'].apply(lambda x: x**3)


df['RH'] = 100 * ( ((17.27 * ((df['dewpnt'] - 32 )/1.8) ) / (
    ((df['dewpnt'] - 32 )/1.8) + 237.3)).apply(math.exp) / (
    (17.27 * ((df['drybulb'] - 32 )/1.8) ) / (
         ((df['drybulb'] - 32 )/1.8) + 237.3)).apply(math.exp) )
df['RH^2'] = df['RH'].apply(lambda x: x**2)
df['RH^3'] = df['RH'].apply(lambda x: x**3)
df = df.drop(columns=['dewpnt'])


df['month'] = df['ts'].apply(lambda x: x.month - 1)
df['week'] = df['ts'].apply(lambda x: x.weekday())
df['hour'] = df['ts'].apply(lambda x: x.hour)

start = pd.Timestamp(2008, 1, 1)
end = pd.Timestamp(2017, 1, 1)
df = df.loc[df['ts'].apply(lambda x: start<= x < end )]

df['S'] = df['month'].apply(lambda x: 1 if x in [6,7,8,9] else 0)


data_path=r'datasets/ISONE2017-2024.csv'

df2 = pd.read_csv(data_path, encoding='gb18030')
df2['ts'] = df2['ts'].apply(lambda x: pd.Timestamp(x) )

# start = pd.Timestamp(2018, 1, 1)
# end = pd.Timestamp(2020, 1, 1)
# df2 = df2.loc[df2['ts'].apply(lambda x: start<= x < end )]

df2['RH'] = 100 * ( ((17.27 * ((df2['dewpnt'] - 32 )/1.8) ) / (
    ((df2['dewpnt'] - 32 )/1.8) + 237.3)).apply(math.exp) / (
    (17.27 * ((df2['drybulb'] - 32 )/1.8) ) / (
         ((df2['drybulb'] - 32 )/1.8) + 237.3)).apply(math.exp) )

df2 = df2.drop(columns=['dewpnt'])

df2['drybulb^2'] = df2['drybulb'].apply(lambda x: x**2)
df2['drybulb^3'] = df2['drybulb'].apply(lambda x: x**3)

df2['RH^2'] = df2['RH'].apply(lambda x: x**2)
df2['RH^3'] = df2['RH'].apply(lambda x: x**3)

df2 = df2[['ts',  'zone', 'demand', 'drybulb', 'RH', 'drybulb^2', 'drybulb^3', 'RH^2', 'RH^3', 'month', 'week', 'hour']]

df2['S'] = df2['month'].apply(lambda x: 1 if x in [6,7,8,9] else 0)

In [4]:

def get_zone_trainMLR(df1,df2, zone, train_start, train_end, valid_start, valid_end, test_start, test_end, log=False, num_hour = 3, num_day = 1, RH = False):
    if zone != 'MASS':
        df_ct1 = df1.groupby('zone').get_group(zone)
        df_ct2 = df2.groupby('zone').get_group(zone)
        df_ct = pd.concat([df_ct1, df_ct2])
    else:
        df_ct = df1.groupby('zone').get_group(zone)
        
    df_ct = df_ct.drop(columns=['zone'])
    df_ct = df_ct.loc[df_ct['ts'].apply(lambda x: train_start - datetime.timedelta(days=240) <= x < test_end )]
    
    df_ct['trend'] = range(1,len(df_ct)+1)
    df_ct = df_ct[['trend', 'ts' , 'demand', 'drybulb','drybulb^2', 'drybulb^3','RH','RH^2','RH^3','S', 'month', 'week', 'hour']] 
        
    df_ct['hour_week'] = df_ct['week'].astype(str) + '_' + df_ct['hour'].astype(str)
    df_ct = pd.get_dummies(df_ct, columns = ['month', 'hour', 'week', 'hour_week'])

    if RH == True:
        for i in range(24):
            cp = df_ct.copy()
            cp['RH_S_hour_{}'.format(i)] = cp['RH'] * cp['S'] * cp['hour_{}'.format(i)]
            cp['RH_S^2_hour_{}'.format(i)] = (cp['RH']**2) * (cp['S']) * cp['hour_{}'.format(i)]
            df_ct = cp
        cp['RH_S'] = cp['RH'] * cp['S']
        cp['RH_S^2'] = (cp['RH']**2) * (cp['S'])
        df_ct = cp
        cp['drybulb_RH_S'] = cp['drybulb'] * cp['RH'] * cp['S']
        cp['drybulb^2_RH_S'] = (cp['drybulb']**2) * cp['RH'] * cp['S']
        df_ct = cp
        cp['drybulb_RH_S^2'] = cp['drybulb'] * cp['S'] * (cp['RH']**2)
        cp['drybulb^2_RH_S^2'] = (cp['drybulb']**2) * cp['S'] * (cp['RH']**2)
        df_ct = cp
        df_ct = df_ct.drop(columns=['RH', 'RH^2', 'RH^3','S'])
        
    else:
        df_ct = df_ct.drop(columns=['RH', 'RH^2', 'RH^3','S'])

    for i in range(12):
        cp = df_ct.copy()
        cp['drybulb_month_{}'.format(i)] = cp['drybulb'] * cp['month_{}'.format(i)]
        cp['drybulb^2_month_{}'.format(i)] = cp['drybulb^2'] * cp['month_{}'.format(i)]
        cp['drybulb^3_month_{}'.format(i)] = cp['drybulb^3'] * cp['month_{}'.format(i)]
        df_ct = cp
    for i in range(24):
        cp = df_ct.copy()
        cp['drybulb_hour_{}'.format(i)] = cp['drybulb'] * cp['hour_{}'.format(i)]
        cp['drybulb^2_hour_{}'.format(i)] = cp['drybulb^2'] * cp['hour_{}'.format(i)]
        cp['drybulb^3_hour_{}'.format(i)] = cp['drybulb^3'] * cp['hour_{}'.format(i)]
        df_ct = cp
        
    for i in range(num_hour):
        df_ct['drybulb_lag{}'.format(i+1)] = df_ct['drybulb'].shift(axis=0, periods=i+1)
        df_ct['drybulb^2_lag{}'.format(i+1)] = df_ct['drybulb^2'].shift(axis=0, periods=i+1)
        df_ct['drybulb^3_lag{}'.format(i+1)] = df_ct['drybulb^3'].shift(axis=0, periods=i+1)
        for p in range(12):
            cp = df_ct.copy()
            cp['drybulb_lag{}_month_{}'.format(i+1, p)] = cp['drybulb_lag{}'.format(i+1)] * cp['month_{}'.format(p)]
            cp['drybulb^2_lag{}_month_{}'.format(i+1, p)] = cp['drybulb^2_lag{}'.format(i+1)] * cp['month_{}'.format(p)]
            cp['drybulb^3_lag{}_month_{}'.format(i+1, p)] = cp['drybulb^3_lag{}'.format(i+1)] * cp['month_{}'.format(p)]
            df_ct = cp
        for q in range(24):
            cp = df_ct.copy()
            cp['drybulb_lag{}_hour_{}'.format(i+1, q+1)] = cp['drybulb_lag{}'.format(i+1)] * cp['hour_{}'.format(q)]
            cp['drybulb^2_lag{}_hour_{}'.format(i+1, q+1)] = cp['drybulb^2_lag{}'.format(i+1)] * cp['hour_{}'.format(q)]
            cp['drybulb^3_lag{}_hour_{}'.format(i+1, q+1)] = cp['drybulb^3_lag{}'.format(i+1)] * cp['hour_{}'.format(q)]
            df_ct = cp
        df_ct = df_ct.drop(columns=['drybulb_lag{}_hour_{}'.format(i+1, q+1)])
        df_ct = df_ct.drop(columns=['drybulb^2_lag{}_hour_{}'.format(i+1, q+1)])
        df_ct = df_ct.drop(columns=['drybulb^3_lag{}_hour_{}'.format(i+1, q+1)])

    for k in range(num_day):
        df_ct['drybulb^a_lag{}'.format(k)] = 0
        for j in range(24):
            df_ct['drybulb^a_lag{}'.format(k)] += df_ct['drybulb'].shift(axis=0, periods= 24 * k + j + 1)

        df_ct['drybulb^a_lag{}'.format(k)] /= 24

        df_ct['drybulb^a^2_lag{}'.format(k)] = df_ct['drybulb^a_lag{}'.format(k)].apply(lambda x: x**2)
        df_ct['drybulb^a^3_lag{}'.format(k)] = df_ct['drybulb^a_lag{}'.format(k)].apply(lambda x: x**3)
        
        for p in range(12):
            cp = df_ct.copy()
            cp['drybulb^a_lag{}_month_{}'.format(k, p)] = cp['drybulb^a_lag{}'.format(k)] * cp['month_{}'.format(p)]
            cp['drybulb^a^2_lag{}_month_{}'.format(k, p)] = cp['drybulb^a^2_lag{}'.format(k)] * cp['month_{}'.format(p)]
            cp['drybulb^a^3_lag{}_month_{}'.format(k, p)] = cp['drybulb^a^3_lag{}'.format(k)] * cp['month_{}'.format(p)]
            df_ct = cp
        for q in range(24):
            cp = df_ct.copy()
            cp['drybulb^a_lag{}_hour_{}'.format(k, q+1)] = cp['drybulb^a_lag{}'.format(k)] * cp['hour_{}'.format(q)]
            cp['drybulb^a^2_lag{}_hour_{}'.format(k, q+1)] = cp['drybulb^a^2_lag{}'.format(k)] * cp['hour_{}'.format(q)]
            cp['drybulb^a^3_lag{}_hour_{}'.format(k, q+1)] = cp['drybulb^a^3_lag{}'.format(k)] * cp['hour_{}'.format(q)]
            df_ct = cp
            
        df_ct = df_ct.drop(columns=['drybulb^a_lag{}'.format(k)])
        df_ct = df_ct.drop(columns=['drybulb^a^2_lag{}'.format(k)])
        df_ct = df_ct.drop(columns=['drybulb^a^3_lag{}'.format(k)])

    # drop unused variables.
    to_drop = ["week_{}".format(i) for i in range(7)
              ] + ["hour_{}".format(i) for i in range(24)] + [
                  "drybulb", "drybulb^2", "drybulb^3"]
    df_ct = df_ct.drop(columns=to_drop)
    
    train_data = df_ct.loc[df_ct['ts'].apply(lambda x: train_start <= x < train_end)]
    valid_data = df_ct.loc[df_ct['ts'].apply(lambda x: valid_start <= x < valid_end)]
    test_data = df_ct.loc[df_ct['ts'].apply(lambda x: test_start <= x < test_end)]
    
    X_train, y_train = train_data.drop(columns=['ts', 'demand']
                                      ).values.copy(), train_data['demand'].values.copy()
    X_valid, y_valid = valid_data.drop(columns=['ts', 'demand']
                                      ).values.copy(), valid_data['demand'].values.copy()
    X_test, y_test = test_data.drop(columns=['ts', 'demand']
                                   ).values.copy(), test_data['demand'].values.copy()
    if log:
        print("zone: {}, \n\t Train start: {}, \n\t Train end: {}".format(zone, train_start, train_end))
        print("zone: {}, \n\t Valid start: {}, \n\t Valid end: {}".format(zone, valid_start, valid_end))
        print("zone: {}, \n\t Test start: {}, \n\t Test end: {}".format(zone, test_start, test_end))
        print("Train set:", X_train.shape)
        print("Valid set:", X_valid.shape)
        print("Test set:", X_test.shape)
    
    return X_train, y_train, X_valid, y_valid, X_test, y_test

In [ ]:
# Tao and RRH

start_year = 2015

Poly_benchmark_dict = {}

for zone in ['CT', 'ME', 'NEMASSBOST', 'NH', 'RI', 'SEMASS','VT', 'WCMASS', 'TOTAL']:

    Poly_benchmark_subdict = {}
    X_train, y_train, X_valid, y_valid, X_test, y_test = get_zone_trainMLR(
        df, df2, zone=zone,
        train_start = pd.Timestamp(start_year, 1, 1),
        train_end = pd.Timestamp(start_year + 2, 1, 1),
        valid_start = pd.Timestamp(start_year + 2, 1, 1),
        valid_end = pd.Timestamp(start_year + 3, 1, 1),
        test_start = pd.Timestamp(start_year + 3, 1, 1),
        test_end = pd.Timestamp(start_year + 4, 1, 1),
        log=True, num_hour = 0, num_day = 0, RH = False
    )

    train_features = X_train.astype(np.float64)
    valid_features = X_valid.astype(np.float64)
    test_features = X_test.astype(np.float64)

    train_labels = y_train.astype(np.float64)
    valid_labels = y_valid.astype(np.float64)
    test_labels = y_test.astype(np.float64)

    [m_train, n_train] = train_features.shape
    [m_valid, n_valid] = valid_features.shape
    [m_test, n_test] = test_features.shape

    X_all = np.concatenate((train_features, valid_features, test_features), axis = 0)
    m = X_all.shape[0]
    mins = np.min(X_all[0:(m_train + m_valid), :], axis = 0)
    maxs = np.max(X_all[0:(m_train + m_valid), :], axis = 0)

    for i in range(m):
        X_all[i,:] = (X_all[i,:] - mins) / (maxs - mins)

    train_features = X_all[:m_train, :]
    valid_features = X_all[m_train:(m_train + m_valid), :]
    test_features = X_all[(m_train + m_valid) :,:]

    # Since the validation set is unnecessary, it is merged with the training set.
    X_train_valid = np.concatenate((train_features, valid_features), axis = 0)
    y_train_valid = np.concatenate((train_labels, valid_labels), axis = 0)

    model = LinearRegression().fit(X_train_valid, y_train_valid)
    y_test_pred = model.predict(test_features)
    preTao = y_test_pred

    test_rmse = RMSE(preTao, y_test)
    test_mape = MAPE(preTao, y_test)
    daily_peak_rmse = RMSE( np.max(preTao.reshape(-1,24) , axis = 1) , np.max(y_test.reshape(-1,24) , axis = 1) )
    daily_peak_mape =  MAPE( np.max(preTao.reshape(-1,24) , axis = 1) , np.max(y_test.reshape(-1,24) , axis = 1) )

    print('zone', zone)
    print('Tao')
    print('RMSE', test_rmse)
    print('Daily Peak MAPE', daily_peak_mape)

    Poly_benchmark_subdict['Tao'] = {
                        "test rmse": test_rmse,
                        "test mape": test_mape,
                        'daily_peak_rmse': daily_peak_rmse,
                        'daily_peak_mape': daily_peak_mape,
                        "test_preds": y_test_pred,
                        "test_trajs": y_test,
                    }
    
    X_train, y_train, X_valid, y_valid, X_test, y_test = get_zone_trainMLR(
        df, df2, zone=zone,
        train_start = pd.Timestamp(start_year, 1, 1),
        train_end = pd.Timestamp(start_year + 2, 1, 1),
        valid_start = pd.Timestamp(start_year + 2, 1, 1),
        valid_end = pd.Timestamp(start_year + 3, 1, 1),
        test_start = pd.Timestamp(start_year + 3, 1, 1),
        test_end = pd.Timestamp(start_year + 4, 1, 1),
        log=True, num_hour = 12, num_day = 2, RH = True
    )

    train_features = X_train.astype(np.float64)
    valid_features = X_valid.astype(np.float64)
    test_features = X_test.astype(np.float64)

    train_labels = y_train.astype(np.float64)
    valid_labels = y_valid.astype(np.float64)
    test_labels = y_test.astype(np.float64)

    [m_train, n_train] = train_features.shape
    [m_valid, n_valid] = valid_features.shape
    [m_test, n_test] = test_features.shape

    X_all = np.concatenate((train_features, valid_features, test_features), axis = 0)
    m = X_all.shape[0]
    mins = np.min(X_all[0:(m_train + m_valid), :], axis = 0)
    maxs = np.max(X_all[0:(m_train + m_valid), :], axis = 0)

    for i in range(m):
        X_all[i,:] = (X_all[i,:] - mins) / (maxs - mins)

    train_features = X_all[:m_train, :]
    valid_features = X_all[m_train:(m_train + m_valid), :]
    test_features = X_all[(m_train + m_valid) :,:]

    # Since the validation set is unnecessary, it is merged with the training set.
    X_train_valid = np.concatenate((train_features, valid_features), axis = 0)
    y_train_valid = np.concatenate((train_labels, valid_labels), axis = 0)

    model = LinearRegression().fit(X_train_valid, y_train_valid)
    y_test_pred = model.predict(test_features)
    preTao = y_test_pred

    test_rmse = RMSE(preTao, y_test)
    test_mape = MAPE(preTao, y_test)
    daily_peak_rmse = RMSE( np.max(preTao.reshape(-1,24) , axis = 1) , np.max(y_test.reshape(-1,24) , axis = 1) )
    daily_peak_mape =  MAPE( np.max(preTao.reshape(-1,24) , axis = 1) , np.max(y_test.reshape(-1,24) , axis = 1) )


    print('zone', zone)
    print('RRH')
    print('RMSE', test_rmse)
    print('Daily Peak MAPE', daily_peak_mape)

    Poly_benchmark_subdict['RRH'] = {
                        "test rmse": test_rmse,
                        "test mape": test_mape,
                        'daily_peak_rmse': daily_peak_rmse,
                        'daily_peak_mape': daily_peak_mape,
                        "test_preds": y_test_pred,
                        "test_trajs": y_test,
                    }
    
    Poly_benchmark_dict[zone] = Poly_benchmark_subdict